### Disclaimer
Please note, the Vantage Functions via SQLAlchemy feature is a preview/beta code release with limited functionality (the “Code”). As such, you acknowledge that the Code is experimental in nature and that the Code is provided “AS IS” and may not be functional on any machine or in any environment. TERADATA DISCLAIMS ALL WARRANTIES RELATING TO THE CODE, EXPRESS OR IMPLIED, INCLUDING, BUT NOT LIMITED TO, ANY WARRANTIES AGAINST INFRINGEMENT OF THIRD-PARTY RIGHTS, MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE.

TERADATA SHALL NOT BE RESPONSIBLE OR LIABLE WITH RESPECT TO ANY SUBJECT MATTER OF THE CODE UNDER ANY CONTRACT, NEGLIGENCE, STRICT LIABILITY OR OTHER THEORY 
    (A) FOR LOSS OR INACCURACY OF DATA OR COST OF PROCUREMENT OF SUBSTITUTE GOODS, SERVICES OR TECHNOLOGY, OR 
    (B) FOR ANY INDIRECT, INCIDENTAL OR CONSEQUENTIAL DAMAGES INCLUDING, BUT NOT LIMITED TO LOSS OF REVENUES AND LOSS OF PROFITS. TERADATA SHALL NOT BE RESPONSIBLE FOR ANY MATTER BEYOND ITS REASONABLE CONTROL.

Notwithstanding anything to the contrary: 
    (a) Teradata will have no obligation of any kind with respect to any Code-related comments, suggestions, design changes or improvements that you elect to provide to Teradata in either verbal or written form (collectively, “Feedback”), and 
    (b) Teradata and its affiliates are hereby free to use any ideas, concepts, know-how or techniques, in whole or in part, contained in Feedback: 
        (i) for any purpose whatsoever, including developing, manufacturing, and/or marketing products and/or services incorporating Feedback in whole or in part, and 
        (ii) without any restrictions or limitations, including requiring the payment of any license fees, royalties, or other consideration. 

In [1]:
# In this notebook, we will be covering examples for following Regular Aggregate Functions
# SQL Documentation: https://docs.teradata.com/reader/756LNiPSFdY~4JcCCcR5Cw/ynL7ziGBufCzuUooZX8VQg
    # 1. BITAND
    # 2. BITNOT
    # 3. BITOR
    # 4. BITXOR
    # 5. COUNTSET
    # 6. GETBIT
    # 7. ROTATELEFT
    # 8. ROTATERIGHT
    # 9. SETBIT
    # 10. SHIFTLEFT
    # 11. SHIFTRIGHT
    # 12. SUBBITSTR
    # 13. TO_BYTE

In [2]:
# Get the connection to the Vantage using create_context()
from teradataml import *
import getpass
td_context = create_context(host=getpass.getpass("Hostname: "), username=getpass.getpass("Username: "), password=getpass.getpass("Password: "))

Hostname: ········
Username: ········
Password: ········


In [3]:
# Load the example dataset.
load_example_data("dataframe", ["bytes_table"])
# Create the DataFrame on 'admissions_train' table
bytes_table = DataFrame("bytes_table")
bytes_table

       byte_col      varbyte_col             blob_col
id_col                                               
0         b'63'      b'627A7863'  b'3330363136323633'
2         b'61'  b'616263643132'  b'6162636431323233'
1         b'62'      b'62717765'  b'3331363136323633'

In [4]:
def print_variables(df, columns):
    print("Equivalent SQL: {}".format(df.show_query()))
    print("\n")
    print(" ************************* DataFrame ********************* ")
    print(df)
    print("\n\n")
    print(" ************************* DataFrame.dtypes ********************* ")
    print(df.dtypes)
    print("\n\n")
    if isinstance(columns, str):
        columns = [columns]
    for col in columns:
        coltype = df.__getattr__(col).type
        if isinstance(coltype, sqlalchemy.sql.sqltypes.NullType):
            coltype = "NullType"
        print(" '{}' Column Type: {}".format(col, coltype))

# Using Bit-Byte Manipulation Functions from Teradata Vanatge with SQLAlchemy

In [5]:
# Import func from SQLAlchemy to use the same for executing bit/byte manipulation functions
from sqlalchemy import func

In [6]:
# Before we move on with examples, one should read below just to understand how teradataml DataFrame and 
# it's columns are used to create a SQLAlchemy ClauseElement/Expression.

# Often in below examples one would see something like this: 'admissions_train.admitted.expression'
# Here in the above expression,
#    'admissions_train' is 'teradataml DataFrame'
#    'admitted' is 'column name' in teradataml DataFrame 'admissions_train'
#    Thus, 
#        'admissions_train.admitted' together forms a ColumnExpression.
#    'expression' allows us to use teradata ColumnExpression to be treated as SQLAlchemy Expression.
#    Thus,
#        'admissions_train.admitted.expression' gives us an expression that can be used with SQLAlchemy clauseElements.

## BITAND Function

In [7]:
# Function performs the logical AND operation on the corresponding bits from the two input arguments.
# Syntax:
# =======
#         BITAND(target_arg, bit_mask_arg)

# Argument Description:
# =====================
#     target_arg
#         A numeric or variable byte expression.
#         Supported types for 'target_arg' are as follows:
#             BYTEINT
#             SMALLINT
#             INTEGER
#             BIGINT
#             DECIMAL
#             NUMBER
#             VARBYTE(n)
#     bit_mask_arg
#         A fixed byte value, a variable byte value, or a numeric expression.

# Function Description:
# =====================
#    This function takes two bit patterns of equal length and performs the logical AND operation on each pair 
#    of corresponding bits as follows:
#        1. If the bits at the same position are both 1, then the result is 1; otherwise, 
#    the result is 0. 
#        2. If either input argument is NULL, the function returns NULL.
#
#    If the target_arg and bit_mask_arg arguments differ in length, the arguments are processed as follows:
#        1. The target_arg and bit_mask_arg arguments are aligned on their least significant byte/bit.
#        2. The smaller argument is padded with zeros to the left until it becomes the same size as the larger argument.

The data type of the bit_mask_arg parameter varies depending upon the data type of the target_arg parameter. 
The following (target_arg, bit_mask_arg) input combinations are permitted:

| target_arg type | bit_mask_arg type |
|------|------|
| BYTEINT	| BYTE(1) |
| BYTEINT	| BYTEINT |
| SMALLINT	| BYTE(2) |
| SMALLINT	| SMALLINT |
| INTEGER	| BYTE(4) |
| INTEGER	| INTEGER |
| BIGINT	| BYTE(8) |
| BIGINT	| BIGINT |
| NUMBER(38,0)	| VARBYTE(16) |
| NUMBER(38,0)	| NUMBER(38,0) |
| VARBYTE(n)	| VARBYTE(n) |

In [8]:
bit_byte_func_ = func.BITAND(bytes_table.id_col.expression, bytes_table.byte_col.expression)
type(bit_byte_func_)

sqlalchemy.sql.functions.Function

In [9]:
# Note: Function name case does not matter. We can use the function name in lower case, upper case or mixed case.
df = bytes_table.assign(True, bitand_id_=bit_byte_func_, 
                        bitand_byte_=func.bitand(bytes_table.varbyte_col.expression, bytes_table.byte_col.expression)
                       )
print_variables(df, ["bitand_id_", "bitand_byte_"])

Equivalent SQL: select bitand(varbyte_col, byte_col) AS bitand_byte_, BITAND(id_col, byte_col) AS bitand_id_ from "bytes_table"


 ************************* DataFrame ********************* 
  bitand_byte_  bitand_id_
0        b'63'           0
1        b'20'           0
2        b'60'           0



 ************************* DataFrame.dtypes ********************* 
bitand_byte_    bytes
bitand_id_        int



 'bitand_id_' Column Type: INTEGER
 'bitand_byte_' Column Type: VARBYTE


## BITNOT Function

In [10]:
# Function performs a bitwise complement on the binary representation of the input argument.
# Syntax:
# =======
#         BITNOT(target_arg)

# Argument Description:
# =====================
#     target_arg
#         A numeric or variable byte expression.
#         Supported types for 'target_arg' are as follows:
#             BYTEINT
#             SMALLINT
#             INTEGER
#             BIGINT
#             VARBYTE(n)

In [11]:
bit_byte_func_ = func.BITNOT(bytes_table.varbyte_col.expression)
type(bit_byte_func_)

sqlalchemy.sql.functions.Function

In [12]:
# Note: Function name case does not matter. We can use the function name in lower case, upper case or mixed case.
df = bytes_table.assign(True, bitnot_byte_=bit_byte_func_)
print_variables(df, ["bitnot_byte_"])

Equivalent SQL: select BITNOT(varbyte_col) AS bitnot_byte_ from "bytes_table"


 ************************* DataFrame ********************* 
       bitnot_byte_
0  b'-616263643133'
1      b'-62717766'
2      b'-627A7864'



 ************************* DataFrame.dtypes ********************* 
bitnot_byte_    bytes



 'bitnot_byte_' Column Type: VARBYTE


## BITOR Function

In [13]:
# Function performs the logical OR operation on the corresponding bits from the two input arguments.
# Syntax:
# =======
#         BITOR(target_arg, bit_mask_arg)

# Argument Description:
# =====================
#     target_arg
#         A numeric or variable byte expression.
#         Supported types for 'target_arg' are as follows:
#             BYTEINT
#             SMALLINT
#             INTEGER
#             BIGINT
#             VARBYTE(n)
#     bit_mask_arg
#         A fixed byte value, a variable byte value, or a numeric expression.

# Function Description:
# =====================
#    This function takes two bit patterns of equal length and performs the logical OR operation on each pair 
#    of corresponding bits as follows:
#        1. If either of the bits from the input arguments is 1, then the result is 1.
#        2. If both of the bits from the input arguments are 0, then the result is 0.
#        3. If any of the input arguments is NULL, then the result is NULL.
#
#    If the target_arg and bit_mask_arg arguments differ in length, the arguments are processed as follows:
#        1. The target_arg and bit_mask_arg arguments are aligned on their least significant byte/bit.
#        2. The smaller argument is padded with zeros to the left until it becomes the same size as the larger argument.

The data type of the bit_mask_arg parameter varies depending upon the data type of the target_arg parameter. 
The following (target_arg, bit_mask_arg) input combinations are permitted:

| target_arg type | bit_mask_arg type |
|------|------|
| BYTEINT	| BYTE(1) |
| BYTEINT	| BYTEINT |
| SMALLINT	| BYTE(2) |
| SMALLINT	| SMALLINT |
| INTEGER	| BYTE(4) |
| INTEGER	| INTEGER |
| BIGINT	| BYTE(8) |
| BIGINT	| BIGINT |
| VARBYTE(n)	| VARBYTE(n) |

In [14]:
bit_byte_func_ = func.Bitor(bytes_table.id_col.expression, bytes_table.byte_col.expression)
type(bit_byte_func_)

sqlalchemy.sql.functions.Function

In [15]:
# Note: Function name case does not matter. We can use the function name in lower case, upper case or mixed case.
df = bytes_table.assign(True, bitor_byte_=bit_byte_func_)
print_variables(df, ["bitor_byte_"])

Equivalent SQL: select Bitor(id_col, byte_col) AS bitor_byte_ from "bytes_table"


 ************************* DataFrame ********************* 
   bitor_byte_
0   1627389954
1   1644167169
2   1660944384



 ************************* DataFrame.dtypes ********************* 
bitor_byte_    int



 'bitor_byte_' Column Type: INTEGER


## BITXOR Function

In [16]:
# Function performs a bitwise XOR operation on the binary representation of the two input arguments.
# Syntax:
# =======
#         BITXOR(target_arg, bit_mask_arg)

# Argument Description:
# =====================
#     target_arg
#         A numeric or variable byte expression.
#         Supported types for 'target_arg' are as follows:
#             BYTEINT
#             SMALLINT
#             INTEGER
#             BIGINT
#             VARBYTE(n)
#     bit_mask_arg
#         A fixed byte value, a variable byte value, or a numeric expression.

# Function Description:
# =====================
#    This function takes two bit patterns of equal length and performs the logical OR operation on each pair 
#    of corresponding bits as follows:
#        1. The result in each position is 1 if the two bits are different.
#        2. The result in each position is 0 if the two bits are same.
#        3. If any of the input arguments is NULL, then the result is NULL.
#
#    If the target_arg and bit_mask_arg arguments differ in length, the arguments are processed as follows:
#        1. The target_arg and bit_mask_arg arguments are aligned on their least significant byte/bit.
#        2. The smaller argument is padded with zeros to the left until it becomes the same size as the larger argument.

The data type of the bit_mask_arg parameter varies depending upon the data type of the target_arg parameter. 
The following (target_arg, bit_mask_arg) input combinations are permitted:

| target_arg type | bit_mask_arg type |
|------|------|
| BYTEINT| BYTE(1) |
| BYTEINT| BYTEINT |
| SMALLINT| BYTE(2) |
| SMALLINT| SMALLINT |
| INTEGER| BYTE(4) |
| INTEGER| INTEGER |
| BIGINT| BYTE(8) |
| BIGINT| BIGINT |
| VARBYTE(n)| VARBYTE(n) |

In [17]:
# Note: Function name case does not matter. We can use the function name in lower case, upper case or mixed case.
df = bytes_table.assign(True, bitxor_int_byte_= func.BITXOR(bytes_table.id_col.expression, bytes_table.byte_col.expression))
print_variables(df, ["bitxor_int_byte_"])

Equivalent SQL: select BITXOR(id_col, byte_col) AS bitxor_int_byte_ from "bytes_table"


 ************************* DataFrame ********************* 
   bitxor_int_byte_
0        1660944384
1        1627389954
2        1644167169



 ************************* DataFrame.dtypes ********************* 
bitxor_int_byte_    int



 'bitxor_int_byte_' Column Type: INTEGER


## COUNTSET Function

In [18]:
# Function returns the count of the binary bits within the target_arg expression 
# that are either set to 1 or set to 0 depending on the target_value_arg value.
# Syntax:
# =======
#         COUNTSET(target_arg, target_arg_value)

# Argument Description:
# =====================
#     target_arg
#         A numeric or variable byte expression.
#     target_value_arg
#         An integer value. Only a value of 0 or 1 is allowed. If target_value_arg is not specified, the default is 1.

# Function Description:
# =====================
#     COUNTSET takes the target_arg input expression and counts the number of bits within the expression 
#     that are either set to 1 or set to 0, depending on the value of target_value_arg.
#
#     The target_value_arg parameter only accepts a value of 0 or 1. If a value for target_value_arg is 
#     not specified, the default value of 1 is used, and COUNTSET counts the bit values that are set to 1.
#
#     If any of the input arguments is NULL, the function returns NULL.

It is defined with the following parameter data types for the following (target_arg [,target_value_arg]) input combinations:

| target_arg type	| target_value_arg type (optional) |
| ------- | ------ |
| BYTEINT |	INTEGER |
| SMALLINT	| INTEGER |
| INTEGER	| INTEGER |
| BIGINT	| INTEGER |
| VARBYTE(n) |	INTEGER |

In [19]:
# Note: Function name case does not matter. We can use the function name in lower case, upper case or mixed case.
df = bytes_table.assign(True, 
                        count_1s_varbyte_= func.COUNTSET(bytes_table.varbyte_col.expression),
                        count_0s_varbyte_= func.COUNTSET(bytes_table.varbyte_col.expression, 0)
                       )
print_variables(df, ["count_1s_varbyte_", "count_0s_varbyte_"])

Equivalent SQL: select COUNTSET(varbyte_col, 0) AS count_0s_varbyte_, COUNTSET(varbyte_col) AS count_1s_varbyte_ from "bytes_table"


 ************************* DataFrame ********************* 
   count_0s_varbyte_  count_1s_varbyte_
0                 16                 16
1                 29                 19
2                 15                 17



 ************************* DataFrame.dtypes ********************* 
count_0s_varbyte_    int
count_1s_varbyte_    int



 'count_1s_varbyte_' Column Type: INTEGER
 'count_0s_varbyte_' Column Type: INTEGER


## GETBIT Function

In [20]:
# Function returns the value of the bit specified by target_bit_arg from the target_arg byte expression.
# Syntax:
# =======
#         GETBIT(target_arg, target_bit_arg)

# Argument Description:
# =====================
#     target_arg
#         A numeric or variable byte expression.
#     target_bit_arg
#         An integer expression.

# Function Description:
# =====================
#     GETBIT gets the bit specified by target_bit_arg from the target_arg byte expression 
#     and returns either 0 or 1 to indicate the value of that bit.
#
#     The range of input values for target_bit_arg can vary from 0 (bit 0 is the least significant bit) 
#     to the (sizeof(target_arg) - 1).
#
#     If target_bit_arg is negative or out-of-range (meaning that it exceeds the size of target_arg), 
#     an error is returned.
#
#     If either input argument is NULL, the function returns NULL.

It is defined with the following parameter data types for the following (target_arg, target_bit_arg) input combinations:

| target_arg type	| target_bit_arg type (optional) |
| ------- | ------ |
| BYTEINT |	INTEGER |
| SMALLINT	| INTEGER |
| INTEGER	| INTEGER |
| BIGINT	| INTEGER |
| VARBYTE(n) |	INTEGER |

In [21]:
# Note: Function name case does not matter. We can use the function name in lower case, upper case or mixed case.
df = bytes_table.assign(True, 
                        getbit_varbyte_= func.GETBIT(bytes_table.varbyte_col.expression, 3),
                        getbit_int_= func.GETBIT(bytes_table.id_col.expression, 1)
                       )
print_variables(df, ["getbit_varbyte_", "getbit_int_"])

Equivalent SQL: select GETBIT(id_col, 1) AS getbit_int_, GETBIT(varbyte_col, 3) AS getbit_varbyte_ from "bytes_table"


 ************************* DataFrame ********************* 
   getbit_int_  getbit_varbyte_
0            0                0
1            1                0
2            0                0



 ************************* DataFrame.dtypes ********************* 
getbit_int_        int
getbit_varbyte_    int



 'getbit_varbyte_' Column Type: BYTEINT
 'getbit_int_' Column Type: BYTEINT


## ROTATELEFT Function

In [22]:
# Function returns an expression rotated to the left by the number of bits you specify, 
# with the most significant bits wrapping around to the right.
# Syntax:
# =======
#         ROTATELEFT(target_arg, num_bits_arg)

# Argument Description:
# =====================
#     target_arg
#         A numeric or variable byte expression.
#     num_bits_arg
#         An integer expression indicating the number of bit positions to rotate.

# Function Description:
# =====================
#     1. If num_bits_arg is equal to zero then the function returns target_arg unchanged.
#     2. If num_bits_arg is negative then the function rotates the bits to the right instead of the left.
#     3. If target_arg and/or num_bits_arg are NULL then the function returns NULL.
#     4. If num_bits_arg is larger than the size of target_arg then the function rotates 
#        (num_bits_arg MOD sizeof(target_arg)) bits. The scope of the rotation operation is bounded by the size
#        of the target_arg expression.
#         NOTE:
#             When operating against an integer value (BYTEINT, SMALLINT, INTEGER, or BIGINT), rotating a bit into the 
#             most significant position will result in the integer becoming negative. This is because all integers in 
#             Teradata Database are signed integers.

It is defined with the following parameter data types for the following (target_arg, num_bits_arg) input combinations:

| target_arg type	| num_bits_arg type (optional) |
| ------- | ------ |
| BYTEINT |	INTEGER |
| SMALLINT	| INTEGER |
| INTEGER	| INTEGER |
| BIGINT	| INTEGER |
| VARBYTE(n) |	INTEGER |

In [23]:
# Note: Function name case does not matter. We can use the function name in lower case, upper case or mixed case.
df = bytes_table.assign(True, 
                        rotateleft_varbyte_= func.ROTATELEFT(bytes_table.varbyte_col.expression, 1),
                        rotateleft_int_= func.ROTATELEFT(bytes_table.id_col.expression, 5)
                       )
print_variables(df, ["rotateleft_varbyte_", "rotateleft_int_"])

Equivalent SQL: select ROTATELEFT(id_col, 5) AS rotateleft_int_, ROTATELEFT(varbyte_col, 1) AS rotateleft_varbyte_ from "bytes_table"


 ************************* DataFrame ********************* 
   rotateleft_int_ rotateleft_varbyte_
0               64    b'-3D3B39379D9C'
1               32        b'-3B1D1136'
2                0        b'-3B0B0F3A'



 ************************* DataFrame.dtypes ********************* 
rotateleft_int_          int
rotateleft_varbyte_    bytes



 'rotateleft_varbyte_' Column Type: VARBYTE
 'rotateleft_int_' Column Type: INTEGER


## ROTATERIGHT Function

In [24]:
# Function returns an expression rotated to the right by the number of bits you specify, 
# with the least significant bits wrapping around to the left.
# Syntax:
# =======
#         ROTATERIGHT(target_arg, num_bits_arg)

# Argument Description:
# =====================
#     target_arg
#         A numeric or variable byte expression.
#     num_bits_arg
#         An integer expression indicating the number of bit positions to rotate.

# Function Description:
# =====================
#     1. If num_bits_arg is equal to zero then the function returns target_arg unchanged.
#     2. If num_bits_arg is negative then the function rotates the bits to the left instead of the right.
#     3. If target_arg and/or num_bits_arg are NULL then the function returns NULL.
#     4. If num_bits_arg is larger than the size of target_arg then the function rotates 
#        (num_bits_arg MOD sizeof(target_arg)) bits. The scope of the rotation operation is bounded by the size
#        of the target_arg expression.
#         NOTE:
#             When operating against an integer value (BYTEINT, SMALLINT, INTEGER, or BIGINT), rotating a bit into the 
#             most significant position will result in the integer becoming negative. This is because all integers in 
#             Teradata Database are signed integers.

It is defined with the following parameter data types for the following (target_arg, num_bits_arg) input combinations:

| target_arg type	| num_bits_arg type (optional) |
| ------- | ------ |
| BYTEINT |	INTEGER |
| SMALLINT	| INTEGER |
| INTEGER	| INTEGER |
| BIGINT	| INTEGER |
| VARBYTE(n) |	INTEGER |

In [25]:
# Note: Function name case does not matter. We can use the function name in lower case, upper case or mixed case.
df = bytes_table.assign(True, 
                        rotateright_varbyte_= func.rotateright(bytes_table.varbyte_col.expression, 1),
                        rotateright_int_= func.ROTATERIGHT(bytes_table.id_col.expression, 2)
                       )
print_variables(df, ["rotateright_varbyte_", "rotateright_int_"])

Equivalent SQL: select ROTATERIGHT(id_col, 2) AS rotateright_int_, rotateright(varbyte_col, 1) AS rotateright_varbyte_ from "bytes_table"


 ************************* DataFrame ********************* 
   rotateright_int_ rotateright_varbyte_
0       -2147483648      b'30B131B21899'
1        1073741824         b'-4EC7444E'
2                 0         b'-4EC2C3CF'



 ************************* DataFrame.dtypes ********************* 
rotateright_int_          int
rotateright_varbyte_    bytes



 'rotateright_varbyte_' Column Type: VARBYTE
 'rotateright_int_' Column Type: INTEGER


## SETBIT Function

In [26]:
# Function sets the value of the bit specified by target_bit_arg to the value of target_value_arg
# in the target_arg byte expression.
# Syntax:
# =======
#         SETBIT(target_arg, target_bit_arg, target_value_arg)

# Argument Description:
# =====================
#    target_arg
#        A numeric or variable byte expression.
#    target_bit_arg
#        An integer expression.
#    target_value_arg
#        An integer value. Only a value of 0 or 1 is allowed. If target_value_arg is not specified, the default is 1.
    
# Function Description:
# =====================
#    SETBIT takes the target_arg input and sets the bit specified by target_bit_arg to the value, 0 or 1, 
#    as provided by the target_value_arg argument.
#
#    The target_value_arg parameter only accepts a value of 0 or 1. If a value for target_value_arg is not specified, 
#    the default value of 1 is used.
#
#    The range of input values for target_bit_arg can vary from 0 (bit 0 is the least significant bit) 
#    to the (sizeof(target_arg) - 1).
#
#    If target_bit_arg is negative or out-of-range (meaning that it exceeds the size of target_arg), an error is returned.
#
#    If any of the input arguments is NULL, the function returns NULL.

It is defined with the following parameter data types for the following (target_arg, target_bit_arg, target_value_arg) input combinations:

| target_arg type	| target_bit_arg type | target_value_arg type(optional) |
| ------- | ------ | ------ |
| BYTEINT |	INTEGER |	INTEGER |
| SMALLINT	| INTEGER |	INTEGER |
| INTEGER	| INTEGER |	INTEGER |
| BIGINT	| INTEGER |	INTEGER |
| VARBYTE(n) |	INTEGER |	INTEGER |

In [27]:
# Note: Function name case does not matter. We can use the function name in lower case, upper case or mixed case.
df = bytes_table.assign(True, 
                        setbit_int_1_1= func.setbit(bytes_table.id_col.expression, 1),
                        setbit_int_2_0= func.SETBIT(bytes_table.id_col.expression, 2, 0)
                       )
print_variables(df, ["setbit_int_1_1", "setbit_int_2_0"])

Equivalent SQL: select setbit(id_col, 1) AS setbit_int_1_1, SETBIT(id_col, 2, 0) AS setbit_int_2_0 from "bytes_table"


 ************************* DataFrame ********************* 
   setbit_int_1_1  setbit_int_2_0
0               2               0
1               2               2
2               3               1



 ************************* DataFrame.dtypes ********************* 
setbit_int_1_1    int
setbit_int_2_0    int



 'setbit_int_1_1' Column Type: INTEGER
 'setbit_int_2_0' Column Type: INTEGER


## SHIFTLEFT Function

In [28]:
# Function returns the expression target_arg shifted by the specified number of bits (num_bits_arg) to the left. 
# The bits in the most significant positions are lost, and the bits in the least significant positions are 
# filled with zeros.
#
# Syntax:
# =======
#         SHIFTLEFT(target_arg, num_bits_arg)

# Argument Description:
# =====================
#    target_arg
#        A numeric or variable byte expression.
#    num_bits_arg
#        An integer expression indicating the number of bit positions to shift.

# Function Description:
# =====================
#     1. If num_bits_arg is equal to zero then the function returns target_arg unchanged.
#     2. If num_bits_arg is negative then the function shifts the bits to the right instead of the left.
#     3. If target_arg and/or num_bits_arg are NULL then the function returns NULL.
#     4. If num_bits_arg is larger than the size of target_arg then the function returns an error.
#        The scope of the shift operation is bounded by the size of the target_arg expression. 
#        Specifying a shift that is outside the range of target_arg results in an SQL error.
#         NOTE:
#             When operating against an integer value (BYTEINT, SMALLINT, INTEGER, or BIGINT), shifting a bit 
#             into the most significant position will result in the integer becoming negative. This is because 
#             all integers in Teradata Database are signed integers.

It is defined with the following parameter data types for the following (target_arg, num_bits_arg) input combinations:

| target_arg type	| num_bits_arg type (optional) |
| ------- | ------ |
| BYTEINT |	INTEGER |
| SMALLINT	| INTEGER |
| INTEGER	| INTEGER |
| BIGINT	| INTEGER |
| VARBYTE(n) |	INTEGER |

In [29]:
# Note: Function name case does not matter. We can use the function name in lower case, upper case or mixed case.
df = bytes_table.assign(True, 
                        shiftleft_varbyte_= func.shiftleft(bytes_table.varbyte_col.expression, 1),
                        shiftleft_int_= func.ShiftLeft(bytes_table.id_col.expression, 2)
                       )
print_variables(df, ["shiftleft_varbyte_", "shiftleft_int_"])

Equivalent SQL: select ShiftLeft(id_col, 2) AS shiftleft_int_, shiftleft(varbyte_col, 1) AS shiftleft_varbyte_ from "bytes_table"


 ************************* DataFrame ********************* 
   shiftleft_int_ shiftleft_varbyte_
0               8   b'-3D3B39379D9C'
1               4       b'-3B1D1136'
2               0       b'-3B0B0F3A'



 ************************* DataFrame.dtypes ********************* 
shiftleft_int_          int
shiftleft_varbyte_    bytes



 'shiftleft_varbyte_' Column Type: VARBYTE
 'shiftleft_int_' Column Type: INTEGER


## SHIFTRIGHT Function

In [30]:
# Function returns the expression target_arg shifted by the specified number of bits (num_bits_arg) to the right. 
# The bits in the least significant positions are lost, and the bits in the most significant positions are filled with zeros.
# Syntax:
# =======
#         SHIFTRIGHT(target_arg, num_bits_arg)

# Argument Description:
# =====================
#    target_arg
#        A numeric or variable byte expression.
#    tnum_bits_arg
#        An integer expression indicating the number of bit positions to shift.

# Function Description:
# =====================
#     1. If num_bits_arg is equal to zero then the function returns target_arg unchanged.
#     2. If num_bits_arg is negative then the function shifts the bits to the left instead of the right.
#     3. If target_arg and/or num_bits_arg are NULL then the function returns NULL.
#     4. If num_bits_arg is larger than the size of target_arg then the function returns an error.
#        The scope of the shift operation is bounded by the size of the target_arg expression. 
#        Specifying a shift that is outside the range of target_arg results in an SQL error.
#         NOTE:
#             When operating against an integer value (BYTEINT, SMALLINT, INTEGER, or BIGINT), shifting a bit 
#             into the most significant position will result in the integer becoming negative. This is because 
#             all integers in Teradata Database are signed integers.

It is defined with the following parameter data types for the following (target_arg, num_bits_arg) input combinations:

| target_arg type	| num_bits_arg type (optional) |
| ------- | ------ |
| BYTEINT |	INTEGER |
| SMALLINT	| INTEGER |
| INTEGER	| INTEGER |
| BIGINT	| INTEGER |
| VARBYTE(n) |	INTEGER |

In [31]:
# Note: Function name case does not matter. We can use the function name in lower case, upper case or mixed case.
df = bytes_table.assign(True, 
                        shiftright_varbyte_= func.shiftright(bytes_table.varbyte_col.expression, 1),
                        shiftright_int_= func.ShiftRight(bytes_table.id_col.expression, 2)
                       )
print_variables(df, ["shiftright_varbyte_", "shiftright_int_"])

Equivalent SQL: select ShiftRight(id_col, 2) AS shiftright_int_, shiftright(varbyte_col, 1) AS shiftright_varbyte_ from "bytes_table"


 ************************* DataFrame ********************* 
   shiftright_int_ shiftright_varbyte_
0                0     b'30B131B21899'
1                0         b'3138BBB2'
2                0         b'313D3C31'



 ************************* DataFrame.dtypes ********************* 
shiftright_int_          int
shiftright_varbyte_    bytes



 'shiftright_varbyte_' Column Type: VARBYTE
 'shiftright_int_' Column Type: INTEGER


## SUBBITSTR Function

In [32]:
# Function extracts a bit substring from the target_arg input expression based on the specified bit position.
#
# Syntax:
# =======
#         SUBBITSTR(target_arg, position_arg, num_bits_arg)

# Arguemnt Description:
# =====================
#    target_arg
#        A numeric or variable byte expression.
#    position_arg
#        An integer expression indicating the starting position of the bit substring to be extracted.
#    num_bits_arg
#        An integer expression indicating the length of the bit substring to be extracted. 
#        This specifies the number of bits for the function to return.

# Function Description:
# =====================
#    SUBBITSTR extracts a bit substring from the target_arg string expression starting at the bit position 
#    specified by position_arg. For the range of bit positions for each data type, see Bit and Byte Numbering Model.
#
#    The num_bits_arg value specifies the length of the bit substring to be extracted and indicates the number
#    of bits that the function should return. Because the return value of the function is a VARBYTE string, 
#    the number of bits returned is rounded to the byte boundary greater than the number of bits requested.
#
#    The bits returned is right-justified, and the excess bits (those exceeding the requested number of bits) 
#    are filled with zeros.
#
#    If position_arg is negative or out-of-range (meaning that it exceeds the size of target_arg), an error is returned.
#
#    If num_bits_arg is negative, or is greater than the number of bits remaining after the starting 
#    position_arg is taken into account, an error is returned.
#
#    If any of the input arguments is NULL, the function returns NULL.

It is defined with the following parameter data types for the following (target_arg, position_arg, num_bits_arg) input combinations:

| target_arg type	| position_arg type | num_bits_arg type |
| ------- | ------ | ------ |
| BYTEINT |	INTEGER |	INTEGER |
| SMALLINT	| INTEGER |	INTEGER |
| INTEGER	| INTEGER |	INTEGER |
| BIGINT	| INTEGER |	INTEGER |
| VARBYTE(n) |	INTEGER |	INTEGER |

In [33]:
# Note: Function name case does not matter. We can use the function name in lower case, upper case or mixed case.
df = bytes_table.assign(True, 
                        subbitstr_varbyte_= func.SUBBITSTR(bytes_table.varbyte_col.expression, 3, 4)
                       )
print_variables(df, ["subbitstr_varbyte_"])

Equivalent SQL: select SUBBITSTR(varbyte_col, 3, 4) AS subbitstr_varbyte_ from "bytes_table"


 ************************* DataFrame ********************* 
  subbitstr_varbyte_
0               b'C'
1               b'6'
2               b'C'



 ************************* DataFrame.dtypes ********************* 
subbitstr_varbyte_    bytes



 'subbitstr_varbyte_' Column Type: VARBYTE


## TO_BYTE Function

In [34]:
# Function converts a numeric data type to the Teradata Database server byte representation (byte value) of the input value.
# Syntax:
# =======
#         TO_BYTE(target_arg)

# Arguemnt Description:
# =====================
#    target_arg
#        A numeric or variable byte expression.
#        It is defined with the following parameter data types:
#           BYTEINT
#           SMALLINT
#           INTEGER
#           BIGINT

# Function Description:
# =====================
#     The number of bytes returned by the function varies according to the data type of the target_arg value.
#
#     If target_arg is NULL, the function returns NULL.

In [35]:
# Note: Function name case does not matter. We can use the function name in lower case, upper case or mixed case.
df = bytes_table.assign(True, 
                        int_to_byte_= func.to_byte(bytes_table.id_col.expression)
                       )
print_variables(df, ["int_to_byte_"])

Equivalent SQL: select to_byte(id_col) AS int_to_byte_ from "bytes_table"


 ************************* DataFrame ********************* 
  int_to_byte_
0         b'2'
1         b'1'
2         b'0'



 ************************* DataFrame.dtypes ********************* 
int_to_byte_    bytes



 'int_to_byte_' Column Type: BYTE


## NOTE: If Incorrect type input is passed to any of these function, one may see following error:
        [Error 9881] Function 'to_byte' called with an invalid number or type of parameters

In [36]:
remove_context()

True